# Table of content
- [ETL Processes tuning](#ETL-Processes-tuning)
    - [Common elements](#Common-elements)
- [Process song_data](#Process-%3Ccode%3Esong_data%3C%2Fcode%3E)
    - [Do some data analysis & quality check](#Do-some-data-analysis-%26amp%3B-quality-check)
        - [Loading all json in a dataframe](#Loading-all-json-in-a-dataframe)
        - [Looking for duplicates](#Looking-for-duplicates)
        - [Choosing SQL data type & constraint](#Choosing-SQL-data-type-%26amp%3B-constraint) 
    - [#1 songs Table](#%231%3A-%3Ccode%3Esongs%3C%2Fcode%3E-Table)
        - [Extract Data for Songs Table](#Extract-Data-for-Songs-Table)
        - [Insert (only one) Record into Song Table](#Insert-(only-one)-Record-into-Song-Table)
        - [Insert all Record into Song Table in one transaction](#Insert-all-Record-into-Song-Table-in-one-transaction)
            - [The morgify() way](#The-morgify()-way)
            - [The COPY way](#The-COPY-way) 
    - [#2: artists Table](#%232%3A-%3Ccode%3Eartists%3C%2Fcode%3E-Table)
        - [Extract Data for Artists Table](#Extract-Data-for-Artists-Table)
        - [Removing the duplicates](#Removing-the-duplicates)
        - [Insert Record into Artist Table](#Insert-Record-into-Artist-Table)
        - [Insert data with copy_from](#Insert-data-with-copy_from)
    - [Lessons learned with the json files](#Lessons-learned-with-the-json-files)
- [Process log_data](#Process-%3Ccode%3Elog_data%3C%2Fcode%3E)
    - [Again... do some data analysis & quality check](#Again...-do-some-data-analysis-%26amp%3B-quality-check)
    - [#3: time Table](#%233%3A-%3Ccode%3Etime%3C%2Fcode%3E-Table)
        - [Extract Data for Time Table](#Extract-Data-for-Time-Table)
        - [Choosing the SQL data type for time table and adding a primary key](#Choosing-the-SQL-data-type-for-time-table-and-adding-a-primary-key)
            - [Why do we need another primary key ?](#Why-do-we-need-another-primary-key-%3F)
        - [Insert Records into Time Table](#Insert-Records-into-Time-Table)
    - [#4: users Table](#%234%3A-%3Ccode%3Eusers%3C%2Fcode%3E-Table)
        - [Extract Data for Users Table](#Extract-Data-for-Users-Table)
        - [Manage duplicates caused by type and keep the last level (or status)](#Manage-duplicates-caused-by-type-and-keep-the-last-level-(or-status))
        - [Choosing the SQL data type for users table](#Choosing-the-SQL-data-type-for-users-table)
        - [Insert Records into Users Table the bulky way](#Insert-Records-into-Users-Table-the-bulky-way)        
    - [#5: songplays Table](#%235%3A-%3Ccode%3Esongplays%3C%2Fcode%3E-Table)
        - [Instructions](#Instructions)
        - [Test the sql query to get song & artist ID](#Test-the-sql-query-to-get-song-%26amp%3B-artist-ID)
        - [Choosing the SQL data type for songplay table](#Choosing-the-SQL-data-type-for-songplay-table)
        - [Getting artists & song ID](#Getting-artists-%26amp%3B-song-ID)
        - [Finally insert songplay data](#Finally-insert-songplay-data)
- [Close Connection to Sparkify Database](#Close-Connection-to-Sparkify-Database)
- [Implement etl.py](#Implement-%3Ccode%3Eetl.py%3C%2Fcode%3E)

# ETL Processes tuning
I used this notebook to develop the ETL process in `etl.py`.<br>
It is useful for understand the reflections made for the data structure, and how data were analysed<br>

## Common elements
This section simply prepare the notebook and the database for further tasks

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [ ]:
%run create_tables.py

In [2]:
# open the database and get a cursor to process SQL queries
conn = psycopg2.connect( DSN_SPARKIFY )
cur = conn.cursor()

In [ ]:
def get_files( root_directory , file_search_query = '*' ):
    '''
        lookup for json file in root_directory and return a list of full file path
        
        Parameters
        ----------
            root_directory : str - filepath
            file_search_query : str - a query string for files, will be appended to each founded folder
        Returns
        -------
            list(str) : a list of full path on each files
        Raise
        -----
            ValueError if filepath is empty  
    '''
        
    if( root_directory == "" ):
        raise ValueError('filepath must be defined')
        
    all_files = []
    #iterate over the folder tree structure
    for root, dirs, files in os.walk( root_directory ):
        #search json files on each folder
        files = glob.glob( os.path.join( root , file_search_query ) )
        for f in files :
            #concatenate result in the list
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, I'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

## Do some data analysis & quality check
I want to get a global impression so I load all datas and use Pandas data frame tools<br>

### Loading all json in a dataframe
I first do a list comprehension to get all content 
```python 
df_from_each_json = ( pd.read_json( f , lines = True ) for f in song_files )
```
but I need to know the file name to help me identify duplicates

In [ ]:
song_files = get_files('data/song_data' , '*.json' )
list_df = [] #list of dataframe

#read each files and append a column with the full filename
for f in song_files:
    df_from_json =  pd.read_json( f , lines = True )
    df_from_json['filename'] = f
    list_df.append( df_from_json )

#concatenate each dataframe in one
df_songs_json_origin = pd.concat( list_df , ignore_index = True )

if(  df_songs_json_origin['artist_id'].count() != len( song_files ) ):
    raise AssertionError('It should have as many row as json files')
else:   
    print('Great : {} rows of data for {} files'.format( len( df_songs_json_origin.index ) , len( song_files ) ) )

### Looking for duplicates
Pandas is a great tool :-) the describe() method help identify quickly duplicates.<br>
Like the ids are alphanumeric (artist_id & song_id), I only include "Object" columns (instead of include='all')<br>
The differnce between count & unique point to some duplication (artist_id & song_id)<br>

In [ ]:
df_songs_json_origin.describe(include=['O'])

Let's detach duplicates with the eponym method from the whole dataset<br>
I have to remove added column (filename) to fit to the original dataset<br>
_note : a previous iteration lead me to know about duplicated files..._

In [ ]:
df_songs_duplicated = df_songs_json_origin[ df_songs_json_origin.drop( 'filename' , axis=1 ).duplicated( keep=False) ]
df_songs_duplicated

So what are these problematic files ?

In [ ]:
df_songs_duplicated['filename'].values

Ok in fact they are no real duplicate I certainly create those file with my direct exploration of data.<br>
But it's a good opportunity to implement and test duplication removal<br>
For that I prefer to use pandas drop_duplicates

In [ ]:
#first remove the filename columns
df_songs_json_origin = df_songs_json_origin.drop( 'filename' , axis=1 )
df_songs_json = df_songs_json_origin.drop_duplicates(inplace=False)
df_songs_json.describe(include='all')

### Choosing SQL data type & constraint
Now my dataset seams to be clean, I have a look at datatype to shape songs & artist tables.

In [ ]:
df_songs_json.dtypes

Nothing really fancy, but I choose (_SQL DATATYPE for numpy datatype_):
- TEXT for object
- NUMERIC for float64
- INT for int64

I choose the most "generic and appropriate" SQL data types.<br>
My aim is to avoid error or bad conversion at import in the ETL pipeline<br>
I also choose artist_id & song_id as primary key in their own table, artist_id in songs table should not be null

## #1: `songs` Table
In this section I will play with different loading technique : simple insert, morgify et COPY.
### Extract Data for Songs Table
Songs table only expose columns song ID, title, artist ID, year, and duration<br>
So I prepare a dataframe to fit this structure.<br>

In [ ]:
df_songs = df_songs_json[['song_id','title','artist_id','year','duration']]
df_songs.values[0]

In [ ]:
df_songs.describe(include='all')

So great : the count & unique on `song_id` confirm the primary key choice.<br>
It seams that only one artist play two songs and each song_id are unique 

### Insert (only one) Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [ ]:
cur.execute( song_table_insert, df_songs.values[0].tolist() )
conn.commit()

_Run `test.ipynb` to see if you've successfully added a record to this table._<br>
...sorry I don't want to leave this page... ;-)

In [ ]:
cur.execute("""SELECT * FROM songs""")
rows = cur.fetchall()
rows

In [ ]:
# do some assertion 
if( df_songs.values[0].tolist()[0] == rows[0][0] ):
    print('Alright, the first row have been inserted with the key {}'.format( df_songs.values[0].tolist()[0] ))
else:
    raise AssertionError('There is a problem with the first cell value, expected {}, actual {}'.format( df_songs.values[0].tolist()[0], rows[0][0] ) )

### Insert all Record into Song Table in one transaction
I my own experience it's better to insert all data in "one shoot" : life is short.<br>
I done some investigation about the COPY command<br>
After some research I found that executemany is inefficient with PostegreSQL (unlike MySQL)<br>
Even psycopg documentation admit that : [Fast execution helpers](http://initd.org/psycopg/docs/extras.html#fast-exec)<br>
I decide to have a look at 'morgify()' and the copy style

#### The morgify() way
So a first solution is to build a large SQL query with morgify() method<br> 
It will construct an insert query with multiple values, like this  `INSERT INTO table_numeric (index) VALUES (1),(2),(3)` <br>
morgify need a parameterized query like `INSERT INTO songs (song_id , title, artist_id, year , duration) VALUES  (%s, %s, %s, %s, %s)`<br>
But it will repeat the whole query again and again. This is why I separate the queries to be able to handle that strings generation (takea a look below)<br>
Some doc [inserting lots of data into a remote Postgres efficiently](https://nelsonslog.wordpress.com/2015/04/27/inserting-lots-of-data-into-a-remote-postgres-efficiently/)<br>

In [ ]:
#to explain my sql_queries organisation
print(' song_table_insert_light  : {}'.format(song_table_insert_light))
print(' song_table_insert_params : {}'.format(song_table_insert_params))
print(' song_table_insert        : {}'.format(song_table_insert))

In [ ]:
# I already inserted the first row, so I sliced the dataframe values... but keep some for copy
data = df_songs.values[1:3].tolist()

dataText = ','.join(cur.mogrify( song_table_insert_params , row).decode('utf-8') for row in data)

print( "The query will be :\n"+song_table_insert_light +"\n"+ dataText )

Great, it's what I'm looking for. Let's execute that query

In [ ]:
# process the insert
cur.execute( song_table_insert_light + dataText )
conn.commit()

In [ ]:
#then query the songs table to display 3 rows
cur.execute("""SELECT * FROM songs""")
rows = cur.fetchall()
rows

In [ ]:
# do some assertion 
if( len(rows) != 3):
    raise AssertionError('The number of row in db is wrong, expected {}, actual {}'.format( 3, len(rows) ) )
elif( df_songs.values[1].tolist()[0] != rows[1][0] ):
    raise AssertionError('There is a problem with the second cell value, expected {}, actual {}'.format( df_songs.values[1].tolist()[0], rows[1][0] ) )
else:
    print('Alright, 3 rows have been inserted with the key {}'.format( df_songs.values[1].tolist()[0] ))

So this method work but we depend on a string variable.<br>
We can only operate in memory and I'm pretty sure that we will have trouble at some point (memory available, size of the input data...)<br>
Let's have a look at the COPY method

#### The COPY way
The COPY method wich is an admin tool of PostgreSQL (https://www.postgresql.org/docs/current/sql-copy.html).<br>
Psycopg implement an interface with `cursor.copy_from()`<br>
I have to pass a StringIO, an in-memory stream<br>

Some links :
- [Using COPY TO and COPY FROM](http://initd.org/psycopg/docs/usage.html#using-copy-to-and-copy-from)
- [`copy_from` documentation](http://initd.org/psycopg/docs/cursor.html#cursor.copy_from)
- [StringIO : An in-memory stream for text I/O](https://docs.python.org/3/library/io.html#io.StringIO)

In [ ]:
#core lib that contains StringIO 
import io 

# create a buffer for CSV infos
buffer = io.StringIO()

# I only want to process the unprocessed
df_songs_the_rest = df_songs[3:].copy()

#serialize the dataframe into the buffer (without index or header)
df_songs_the_rest.to_csv(buffer , index=False , header=False)

# I have the move the pointer at the start of the stream to do another iteration
buffer.seek(0)

#default params doesn't fit to CSV
cur.copy_from(buffer, 'songs' , sep=',' , columns=( 'song_id', 'title', 'artist_id', 'year', 'duration') )

In [ ]:
#check if the `copy_from` works
cur.execute("""SELECT * FROM songs""")
rows = cur.fetchall()

In [ ]:
# do some assertion 
if( len(rows) != 71):
    raise AssertionError('The number of row in db is wrong, expected {}, actual {}'.format( 71, len(rows) ) )
else:
    print('\n\nAlright, 71 rows have been inserted with the key {}\n'.format( df_songs.values[1].tolist()[0] ))

Ok I'm happy with both method, I will use mogrify() to process small amount of data (in this notebook) and copy_from for the ETL

## #2: `artists` Table
The previous section defined the main technique that I will used for the ETL pipeline<br>
I will care about data only
### Extract Data for Artists Table
Here again i will only choose a subset of columns : artist ID, name, location, latitude, and longitude

In [ ]:
df_artist_origin = df_songs_json[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
df_artist_origin .head(3)

In [ ]:
df_artist_origin.describe(include='all')

#### Removing the duplicates
Here my artist_id key has duplcate, first have a look on duplication and remove those rows

In [ ]:
#first identify concerned rows
df_artist_duplicated = df_artist_origin[ df_artist_origin.duplicated( keep=False) ]
df_artist_duplicated

ok, no doubts, I can remove thos rows

In [ ]:
df_artist = df_artist_origin.drop_duplicates(inplace=False)
df_artist.describe(include='all')

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute(artist_table_insert, df_artist.values[0].tolist() )
conn.commit()

In [ ]:
#then query the artist table to display 3 rows
cur.execute("""SELECT * FROM artists""")
rows = cur.fetchall()
rows

Run `test.ipynb` to see if you've successfully added a record to this table.

### Insert artist data with copy_from
To make COPY works I had to manage several points :
- uniformize empty value in order to consider them in copy_from()
- define another separator (tab) in the CSV file thus I can import strings with comma

In [ ]:
#replace NaN of longitude / lattitude with a arbitrary NULL value
df_artist = df_artist.fillna('NULL')
#replace all empty cells with NULL
df_artist= df_artist.replace('','NULL')
#avoid usage of tab in all cells...
df_artist = df_artist.replace('\t','')

In [ ]:
# I only want to process the unprocessed
df_artist_the_rest = df_artist[1:].copy()

# create a buffer for CSV infos
buffer = io.StringIO()

#serialize the dataframe into the buffer (without index or header and specify tab as separator)
df_artist_the_rest.to_csv( buffer , index=False , header=False, sep='\t' )

# I have the move the pointer at the start of the stream to do another iteration
buffer.seek(0)

# do it in artist with tab as separator
cur.copy_from(buffer, 'artists' , sep='\t', null='NULL' , columns=( 'artist_id', 'name', 'location', 'lattitude', 'longitude' ) )

In [ ]:
#query the whole table
cur.execute("""SELECT * FROM artists""")
rows = cur.fetchall()

In [ ]:
# do some assertion 
if( len(rows) != len( df_artist.index ) ):
    raise AssertionError('The number of row in db is wrong, expected {}, actual {}'.format( len( df_artist.index ) , len(rows) ) )
else:
    print('\n\n Alright, {} artists have been inserted\n'.format(len(rows)))

## Lessons learned with the json files
1. In my case, I have to deal with duplicates files.. they were created by my own with the jupyter lab :-)<br>
2. I tested multiple methods and focused on heavy load method COPY<br>
Working with the whole set of data, instead of one row, allow to quickly identify a lot of problem<br>
To make it work, as far as I can see, I have to take care of NULL<br>
3. The PostgreSQL implementation of COPY with CSV is limited to a one-byte character.<br>
I choose tabulation for this purpose because there are no "rich" textual informations and little chance to meet<br>

# Process `log_data`
In this part, I perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

## Again... do some data analysis & quality check
With my first iteration with songs_data, I directly suspect duplicates on files but there wasn't the case (I cannot open log_data in Jupyter ;-).<br>
Anyway I load content in a dataframe and try to remove duplicate lines<br>
The log seems pretty good, the reason is certainly because it come from a server generated log. Those above come from human input.. trust the machine ;-)


In [ ]:
log_files = get_files('data/log_data' , '*.json' )

#load each files in a data frame
df_logs_json = ( pd.read_json( f , lines = True ) for f in log_files )

#concatenate each dataframe in one
df_logs_origin = pd.concat( df_logs_json , ignore_index = True )

#drop duplicates
df_logs = df_logs_origin.drop_duplicates(inplace=False)

print( '\n\nThere is {} duplicate\n'.format( len(df_logs.index) - len(df_logs_origin.index)) )

I directly create a dataframe with NextSong only.<br>
The describe(all) show that a log file is well filled with the same `count` for each columns<br>
So I don't need to wrangle data :-)

In [ ]:
# filter the content to page NextSong and create a new copy of the dataframe
df_NextSong = df_logs[ df_logs['page'] == 'NextSong' ].copy()
df_NextSong.describe(include='all')

## #3: `time` Table
### Extract Data for Time Table
>_base instructions :_
>- Filter records by `NextSong` action
>- Convert the `ts` timestamp column to datetime
>  - Hint: the current timestamp is in milliseconds
>- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these >values in order
>  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
>- Specify labels for these columns and set to `column_labels`
>- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

I do all above instructions directly in a dataframe<br>
`df_NextSong` already adress NextSong log entries (necessity of previous section)<br>
I add all new columns to the base dataframe to facilitate db load

In [ ]:
#convert the ts column to a datetime column
df_NextSong['start_time'] = pd.to_datetime( df_NextSong['ts'], unit='ms')

#append column with the correct name
df_NextSong['hour'] = df_NextSong['start_time'].dt.hour
df_NextSong['day'] = df_NextSong['start_time'].dt.day
df_NextSong['week'] = df_NextSong['start_time'].dt.week
df_NextSong['month'] = df_NextSong['start_time'].dt.month
df_NextSong['year'] = df_NextSong['start_time'].dt.year
df_NextSong['weekday'] = df_NextSong['start_time'].dt.weekday

copy_from() doesn't work at all with a timestamp including milliseconds<br>
It seems that the momentum is fixed on .769 \[ms\] on every row... so I decide to remove this glitch with a tape conversion

In [ ]:
#remove the ms part of datetime with a conversion
df_NextSong['start_time'] = df_NextSong['start_time'].astype('datetime64[s]')

In [ ]:
# create the time table dataframe
df_times = df_NextSong[['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']]
df_times.head(3)

### Choosing the SQL data type for time table and adding a primary key

In [ ]:
df_times.dtypes

dTypes lead me simply to `int`<br>
For `start_time` the format looks like PostgreSQL `TIMESTAMP`<br>

#### Why do we need another primary key ?
`time` & `songs_play` table are in a 1:1 relation (linked by the log row)<br>
But there are "only" 6813 unique start_time on 6820.<br>
It means that some transactions occurs at the same time<br>   
To avoid 1:N association, I have to add an extra column `time_id` which will be the index of the dataframe<br>

In [ ]:
df_times.describe(include='datetime64')

Of course, we have to to look in the main dataframe to prove the synchronicity of transaction.<br> 
We can see below that those duplicate rows are clearly separate transaction and clearly consecutive<br>

_Thanks to [DSM on stackOverflow](https://stackoverflow.com/questions/14657241/how-do-i-get-a-list-of-all-the-duplicate-items-using-pandas-in-python) for the query below..._

In [ ]:
#I limit the columns output to make it more readable
pd.concat(g for _, g in df_NextSong.groupby("start_time") if len(g) > 1)[['start_time','artist','firstName','lastName','userId']]

### Insert Records into Time Table
> _original instructions_ :<br>
> Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.
>```python
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()
```    

So here I will directly implement the COPY method.<br>
The big deal here was to understand from where the problem with copy_from()<br>
There was no error output, so I have to found that PostgreSQL `TIMESTAMP` doesn't like Pandas `datetime64`, in particular the millisecond part<br>
My solution was to convert `start_time` with `df_NextSong['start_time'].astype('datetime64[s]')` to remove the \[ms\] part ([look above](#Extract-Data-for-Time-Table))<br>
A simpler solution could be to store start_time as TEXT in the DB... but less fun<br>

In [ ]:
# create a buffer for CSV infos
buffer = io.StringIO()

#serialize the dataframe into the buffer with index as a key
df_times.to_csv(buffer , index=True , header=False , sep='\t' )

# I have the move the pointer at the start of the stream to do another iteration
buffer.seek(0)

#default params doesn't fit to CSV
cur.copy_from(buffer, 'times' , sep='\t' , columns=( 'time_id', 'start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday') )

In [ ]:
cur.execute("""SELECT * FROM times""")
rows = cur.fetchall()

In [ ]:
# do some assertion 
if( len(rows) != len( df_times.index ) ):
    raise AssertionError('The number of row in db is wrong, expected {}, actual {}'.format( len( df_times.index ) , len(rows) ) )
else:
    print('\n\n Alright, {} times rows have been inserted\n'.format(len(rows)))

## #4: `users` Table
### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `df_users`

In [ ]:
df_all_users = df_NextSong[['userId', 'firstName', 'lastName', 'gender', 'level']]
df_all_users.head(3)

### Manage duplicates caused by type and keep the last level (or status)
I can see that there are lot of duplicates. Instructions state that songplays use user ID as a key<br>
After some trial, I found that userId can be either `str` (when level = paid) or `int` (when level = free) !<br>
That lead to some drop_duplicates() trouble<br>
_Note : I can report the problem about log function to the dev team ;-)_

In [ ]:
#copy df to be able to change the content of `user_df`
df_users = df_all_users.copy()

#convert userID to int to 
df_users['userId'] = pd.to_numeric( df_users['userId'])
df_users.drop_duplicates(inplace=True)
df_users.describe( include='all' )

In [ ]:
print( 'On {} rows, only {} are unique'.format( df_users['userId'].count() , len(df_users['userId'].unique()) ) )

`df_users` doesn't have empty cells, nice<br>
But the difference between userID count (104) and unique (96) has to be investigated<br>
Again I reuse the "famous" DSM query to extract "groupedby userID" greater then 1<br>

In [ ]:
pd.concat(g for _, g in df_users.groupby("userId") if len(g) > 1)

I investigate the evolution of level in the base log data

In [ ]:
#extract only usefull info 
df_investigate = df_NextSong[ df_NextSong["userId"].astype(str) == "88" ][['userId','level','start_time','artist']]

#put first & last 3 rows
pd.concat( [ df_investigate.head(3) , df_investigate.tail(3) ] )

I'm not pretty sure what is the meaning of `level` column in a `users` perspective<br>
By the way this info will be accessible in `songplays` tuple associate with the user<br>
I interpret it like the "current status" of the enrollment<br>
So I decide to keep this column but keep only the last value

In [ ]:
# avoid `level` in drop_duplicates()process and keep only the last one
df_users.drop_duplicates( inplace=True, keep='last' , subset=['userId', 'firstName', 'lastName', 'gender'])

#confirm the duplication eradication
print( '\n\nNow, on {} rows, {} are unique\n'.format( df_users['userId'].count() , len( df_users['userId'].unique() ) ) )

## Managing duplicate with PostgreSQL `Upsert`
The first review point me to the `ON CONFLICT` statment that can be really usefull<br>
As I clean data in python (and I have to because of copy_from() usage), I could state that `ON CONFLICT DO NOTHING` is enough.. but nothing to learn<br>
First create a use case with duplicates :

In [ ]:
# will contains a list of identical users expect on the level fields
test_users = []

# arbitrary get the first users in the df
first_user = df_users.values[0].tolist()

# append seamly users (by copy!)
test_users.append( first_user.copy() )
first_user[4] = 'paid1'
test_users.append(first_user.copy() )
first_user[4] = 'paid2'
test_users.append(first_user.copy() )
test_users

In [ ]:
# append statment to manage conflict
user_table_insert_conflict = user_table_insert +' ON CONFLICT(user_id) DO UPDATE set level=EXCLUDED.level'
#print( user_table_insert_conflict )
user_table_insert

In [ ]:
# do the insert several times without error
for user in test_users:
    cur.execute( user_table_insert , user )
    conn.commit()

In [ ]:
cur.execute('SELECT * FROM users WHERE users.user_id = {}'.format( first_user[0] ) )
rows = cur.fetchall()
rows

In [ ]:
# rows[0]
# do some assertion 
if( len(rows) != 1 ):
    raise AssertionError('The number of row in db is wrong, expected {}, actual {}'.format( 1  , len(rows) ) )
elif( rows[0][4] != first_user[4]):
    raise AssertionError('Upsert doesn\'t with the level, expected {}, actual {}'.format( first_user[4] , rows[0][4]  ) )
else:
    print('\n\n Alright, level have been changed to {} \n'.format(rows[0][4] ) )

In [ ]:
# delete the row for the above processing
cur.execute( 'DELETE FROM users WHERE users.user_id = {}'.format( first_user[0] ) )

As usual, ther is a good tutorial on the eponym site : [PostgreSQL Upsert Using INSERT ON CONFLICT statement](http://www.postgresqltutorial.com/postgresql-upsert/) <br>
> as stated above, because of the usage of copy_from() and mass loading with CSV file : I have to care about duplicate before loading

### Choosing the SQL data type for users table
Nothing fancy here just TEXT fields.<br>
I can convert `userID` to `INT` to make it a `PRIMARY KEY` but should keep in mind the types mix on `songsplay`

In [ ]:
df_users.dtypes

### Insert Records into Users Table the bulky way
> _original instructions_ :<br>
> Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.
> ```python
>for i, row in user_df.iterrows():
>     cur.execute(user_table_insert, row)
>     conn.commit()
> ```

In [ ]:
# create a buffer for CSV infos
buffer = io.StringIO()

#serialize the dataframe into the buffer
df_users.to_csv(buffer , index=False , header=False , sep='\t' )

# move the pointer at the start of the stream to let copy_from do its job
buffer.seek(0)

#do the export
cur.copy_from(buffer, 'users' , sep='\t' , columns=('user_id', 'first_name', 'last_name', 'gender', 'level') )

In [ ]:
cur.execute("""SELECT * FROM users""")
rows = cur.fetchall()

In [ ]:
# do some assertion 
if( len(rows) != len( df_users.index ) ):
    raise AssertionError('The number of row in db is wrong, expected {}, actual {}'.format( len( df_users.index ) , len(rows) ) )
else:
    print('\n\n Alright, {} users have been inserted\n'.format(len(rows)))

## #5: `songplays` Table
### Instructions
**Extract Data and Songplays Table**

This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

**Insert Records into Songplays Table**
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

> Here is the original function proposed to insert songplay which isn't compatible with COPY<br>

```python
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
```

### Test the sql query to get song & artist ID
I will try to retrieve a specific entry in `df_songs` to confirm that my query is correct

In [ ]:
# extract the first entry as a list : 
# the first 3 elements are the parameters
# the last 2 are the expected results
test_song = list(df_songs_json[['artist_name','title','duration','song_id','artist_id']].loc()[0])
test_song 

In [ ]:
song_select_for_test = ("""
SELECT 
     songs.song_id, artists.artist_id
FROM 
    songs 
INNER JOIN 
    artists ON (artists.artist_id = songs.artist_id)
WHERE 
        artists.name = %s
    and songs.title = %s 
    and songs.duration = %s 
""")

# the first 3 elements are the parameters
cur.execute( song_select_for_test , test_song[0:3] )
results = cur.fetchone()

In [ ]:
# do some assertion 
if( results != tuple(test_song[3:]) ):
    raise AssertionError('The SQL request is , expected {}, actual {}'.format( tuple(test_song[3:]) , results ) )
else:
    print('\n\n Alright, the query return the ids : {} which is correct\n'.format(results))

### Choosing the SQL data type for songplay table
I will create a new dataframe to
- reduce the number of columns
- convert the userId columns
- add a column with an index ( keep in mind the 1:1 relation with times table)

In [ ]:
#copy the avaialble column to a new dataframe
df_songplay = df_NextSong[['userId', 'level', 'sessionId',  'location', 'userAgent','artist','song','length']].copy()

#remeber the bug between free & paid log detected in users table
df_songplay['userId'] = pd.to_numeric( df_songplay['userId'])
df_songplay['time_id'] = df_songplay.index

df_songplay.describe(include ='all')

No problem with duplicate, all columns present 6820 entries<br>
Of course song_id, artist_id should match there foreign key type (TEXT)

### Getting artists & song ID
The initial version consist on a for loop with directs INSERT in db (see instructions)<br>
I want to implement the COPY method so I have to integrate this info in `df_songplays` to allow usage of to_csv() (asnd for fun ;-)<br>
But I have slightly problem : my songs data is really poor and neither songs is present in log data !.<br>
I want to be able to test my code : So first I have to distribute songs data on logs data...

In [ ]:
# assign knows values to df_songplay
for index, row in df_songs_json[:10].iterrows():
    df_songplay.loc[index,'artist'] =row['artist_name']
    df_songplay.loc[index,'song'] = row['title']
    df_songplay.loc[index,'length'] = row['duration']
    
df_songplay.head(5)

I want to take advantage on the concise notation of Pandas through apply()<br>
So I implement a lambda that will lookup infos in the db

In [ ]:
song_select_for_test_2 = ("""
SELECT 
     songs.song_id, artists.artist_id
FROM 
    songs 
INNER JOIN 
    artists ON (artists.artist_id = songs.artist_id)
WHERE 
        artists.name = %s
    and songs.title = %s 
    and songs.duration = %s
""")

def lookup_song_and_artist( params , cur , query ):
    '''
        >> Function to apply in a dataframe that came from a Log Dataset <<
        Based on params[], query the DB for artist & song ID
        Parameters
        ----------
            It is tricky to use to because params must be well ordered
            params[0] : str - artist name
            params[1] : str - song title
            params[2] : decimal - length of songs
            cur : cursor - the psycopg cursor used to trigger the query
            query : str - the sql query to execute with params (to be passed bay apply(args=) )
        Returns
        -------
            list(songid, artistid) : 
                - a list of related entity db primary key
                - 'NULL', 'NULL' if nothing found
                - 'Error' , 'exception error' in case of exceptions
    '''
    #concretise params
    artist = params[0]
    song = params[1]
    length = params[2]
        
    
    try:
        # query the db
        cur.execute( query , (artist, song , length))
        results = cur.fetchone()
    except psycopg2.Error as e:
        # catch the error and return an empty result
        results = 'Error' , e 
        
    if results:
        #ok we have a match or an error
        songid, artistid = results
    else:
        #instead we nulls
        songid, artistid = 'NULL', 'NULL'
        
    return ( songid, artistid )  


df_songplay['songid'],  df_songplay['artistid'] = zip(*df_songplay[['artist','song' , 'length' ]].apply( lookup_song_and_artist , axis=1 , args=(cur, song_select_for_test_2,)))

df_songplay[df_songplay['songid'] != 'NULL'].head()

### Finally insert songplay data
Like other table, I will implement copy_from()<br>
I have to rearrange `df_songplay` column to simplify the to_csv() export

In [ ]:
df_songplay = df_songplay[['time_id','userId','level','songid', 'artistid','sessionId','location','userAgent']]

In [ ]:
# create a buffer for CSV infos
buffer = io.StringIO()

#serialize the dataframe into the buffer
df_songplay.to_csv(buffer , index=True , header=False , sep='\t' )

# move the pointer at the start of the stream to let copy_from do its job
buffer.seek(0)
()
#do the export
cur.copy_from(buffer, 'songplays' , sep='\t' , \
              columns=('songplay_id', 'time_id', 'user_id', 'level', 'song_id', 'artist_id', 'session_id', 'location', 'user_agent') )

In [ ]:
cur.execute("""SELECT * FROM songplays""")
rows = cur.fetchall()

In [ ]:
# do some assertion 
if( len(rows) != len( df_songplay.index ) ):
    raise AssertionError('The number of row in db is wrong, expected {}, actual {}'.format( len( df_songplay.index ) , len(rows) ) )
else:
    print('\n\n Alright, {} songplay have been inserted\n'.format(len(rows)))

# asking some questions
OK so normally the full ETL pipeline has processed so the db is full of datas<br>
I used this section to develop my dashbord queries (see `dashboard.ipynb`)

In [37]:
query_count_songplay_by_location_test ="""
SELECT 
     songplays.location
    , COUNT(songplays.songplay_id) as songplay_counter
    , COUNT(songplays.songplay_id)::decimal / {tot} * 100
FROM 
    songplays
GROUP BY
    songplays.location 
ORDER BY 
    songplay_counter DESC
"""
query_count_songplay_by_location_test.format(tot=6820)

'\nSELECT \n     songplays.location\n    , COUNT(songplays.songplay_id) as songplay_counter\n    , COUNT(songplays.songplay_id)::decimal / 6820 * 100\nFROM \n    songplays\nGROUP BY\n    songplays.location \nORDER BY \n    songplay_counter DESC\n'

In [39]:
cur.execute(query_count_songplay_by_location_test.format(tot=6820))
rows = cur.fetchall()
print( len(rows) )
rows[:10]

63


[('San Francisco-Oakland-Hayward, CA',
  691,
  Decimal('10.13196480938416422300')),
 ('Portland-South Portland, ME', 665, Decimal('9.75073313782991202300')),
 ('Lansing-East Lansing, MI', 557, Decimal('8.16715542521994134900')),
 ('Chicago-Naperville-Elgin, IL-IN-WI',
  475,
  Decimal('6.96480938416422287400')),
 ('Atlanta-Sandy Springs-Roswell, GA', 456, Decimal('6.68621700879765395900')),
 ('Waterloo-Cedar Falls, IA', 397, Decimal('5.82111436950146627600')),
 ('Lake Havasu City-Kingman, AZ', 321, Decimal('4.70674486803519061600')),
 ('Tampa-St. Petersburg-Clearwater, FL',
  307,
  Decimal('4.50146627565982404700')),
 ('San Jose-Sunnyvale-Santa Clara, CA',
  292,
  Decimal('4.28152492668621700900')),
 ('Sacramento--Roseville--Arden-Arcade, CA',
  270,
  Decimal('3.95894428152492668600'))]

In [30]:
# Close Connection to Sparkify Database
conn.close()
# conn = psycopg2.connect( DSN_SPARKIFY )
# cur = conn.cursor()